# This notebook tries to improve how well the model describes the reality by adjusting it's weights

In [4]:
from python import *

import pickle
import os
import numpy as np
import json
from ipyparallel import Client
from itertools import repeat

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [5]:
#Create data classes
#Year, Season
seasonsData = None
#RaceId, List of tuples of (driverId, constructorId, time)
qualiResultsData = None
#DriverId, name
driversData = None
#ConstructorId, name
constructorsData = None
#EngineId, name
enginesData = None

with open('data/seasonsData.txt', 'rb') as handle:
    seasonsData = pickle.load(handle)
    #print(seasonsData)
    
with open('data/qualiResultsData.txt', 'rb') as handle:
    qualiResultsData = pickle.load(handle)
    #print(qualiResultsData)
    
with open('data/driversData.txt', 'rb') as handle:
    driversData = pickle.load(handle)
    #print(driversData)
    
with open('data/constructorsData.txt', 'rb') as handle:
    constructorsData = pickle.load(handle)
    #print(constructorsData)
    
with open('data/enginesData.txt', 'rb') as handle:
    enginesData = pickle.load(handle)
    #print(enginesData)

In [6]:
entries = []
results = []
cleaner = F1DataCleaner(seasonsData, qualiResultsData, driversData, constructorsData, enginesData)

#Constants we can change

cleaner.k_engine_change = 0.0145
cleaner.k_const_change = 0.240
cleaner.k_driver_change = 0.19
cleaner.k_const_impact = 0.80
cleaner.k_eng_impact = (1 - cleaner.k_const_impact)
cleaner.k_driver_impact = 0.02
        
cleaner.k_rookie_pwr = 0.40
#cleaner.k_rookie_variance = 5
cleaner.k_race_regress_exp = 0.87  #TODO needs to change!
#cleaner.k_variance_multiplier_end = 1.5

cleaner.k_eng_regress = 1.04
cleaner.k_const_regress = 0.60
cleaner.k_driver_regress = 0.67

cleaner.constructDataset(entries, results)
#print(entries[-10:])
#print(results[-10:])
X = np.array(entries)
y = results

forest = RandomForestRegressor(random_state=0, n_estimators=100)
forest.fit(X, y)
print(forest.feature_importances_)
#Rates based on test data!
print(forest.score(X, y))

[0.15264976 0.584018   0.09514376 0.04745465 0.05304647 0.06768737]
0.9413309942662423


In [7]:
#Linear regression
reg = LinearRegression().fit(X, y)
print("R-score: " + str(reg.score(X, y)))
print(reg.coef_)

R-score: 0.5358747645772608
[0.52531956 0.83068053 0.67899941 0.1018996  0.20121132 2.25150429]


In [8]:
#Random forest regression

forest = RandomForestRegressor(max_depth=5, random_state=0,
                             n_estimators=100)
forest.fit(X, y)
print(forest.feature_importances_)
#Rates based on test data!
print(forest.score(X, y))

[0.10967272 0.82030261 0.03463206 0.00349515 0.00614702 0.02575045]
0.6288792074045544


# Just for testing!

In [9]:
newDrivers = json.load(open('data/newDrivers.json'))["drivers"]
newDrivers = {int(did): cid for did, cid in newDrivers.items()}

#Manual changes
#cleaner.drivers[8].constructor = cleaner.constructors[15] #Kimi
#cleaner.drivers[841].constructor = cleaner.constructors[15] #Gio
#cleaner.drivers[844].constructor = cleaner.constructors[6] #Leclerc
#cleaner.drivers[1000] = Driver("Lando Norris", cleaner.constructors[1]) #Lando
#cleaner.drivers[832].constructor = cleaner.constructors[1] #Sainz
#cleaner.drivers[840].constructor = cleaner.constructors[10] #Stroll
#cleaner.drivers[842].constructor = cleaner.constructors[9] #Gasly
#cleaner.drivers[817].constructor = cleaner.constructors[4] #Ricciardo
#cleaner.drivers[1001] = Driver("Alexander Albon", cleaner.constructors[5]) #Albon
#cleaner.drivers[826].constructor = cleaner.constructors[5] #Kvyat
#cleaner.drivers[1002] = Driver("George Russell", cleaner.constructors[3]) #Russell
#cleaner.drivers[9].constructor = cleaner.constructors[3] #Kubica

#cleaner.drivers[1000].pwr = cleaner.k_rookie_pwr
#cleaner.drivers[1001].pwr = cleaner.k_rookie_pwr
#cleaner.drivers[1002].pwr = cleaner.k_rookie_pwr

driversToWrite = {}
for did, cid in newDrivers.items():
    driversToWrite[int(did)] = {}
    driversToWrite[int(did)]["name"] = cleaner.drivers[int(did)].name
    driversToWrite[int(did)]["constructor"] = cleaner.drivers[int(did)].constructor.name
print(driversToWrite)
with open('../F1PredictWeb/public/driversDescribe.json', 'w') as fp:
    json.dump(driversToWrite, fp)

{8: {'name': 'Kimi Räikkönen', 'constructor': 'Alfa Romeo'}, 841: {'name': 'Antonio Giovinazzi', 'constructor': 'Alfa Romeo'}, 20: {'name': 'Sebastian Vettel', 'constructor': 'Ferrari'}, 844: {'name': 'Charles Leclerc', 'constructor': 'Ferrari'}, 154: {'name': 'Romain Grosjean', 'constructor': 'Haas F1 Team'}, 825: {'name': 'Kevin Magnussen', 'constructor': 'Haas F1 Team'}, 846: {'name': 'Lando Norris', 'constructor': 'McLaren'}, 832: {'name': 'Carlos Sainz', 'constructor': 'McLaren'}, 1: {'name': 'Lewis Hamilton', 'constructor': 'Mercedes'}, 822: {'name': 'Valtteri Bottas', 'constructor': 'Mercedes'}, 815: {'name': 'Sergio Pérez', 'constructor': 'Racing Point'}, 840: {'name': 'Lance Stroll', 'constructor': 'Racing Point'}, 842: {'name': 'Pierre Gasly', 'constructor': 'Red Bull'}, 830: {'name': 'Max Verstappen', 'constructor': 'Red Bull'}, 817: {'name': 'Daniel Ricciardo', 'constructor': 'Renault'}, 807: {'name': 'Nico Hülkenberg', 'constructor': 'Renault'}, 848: {'name': 'Alexander Al

In [10]:
circuit = 4
circuitName = "Spanish"

with open('../F1PredictWeb/public/circuitName.txt', 'w') as fp:
    fp.write(circuitName);

In [11]:
predictedEntrants = []

for did, cid in newDrivers.items():
    print(cleaner.drivers[did].name + ": " + str(cleaner.drivers[did].pwr) + " -- " + str(cleaner.drivers[did].constructor.pwr))
    if circuit not in cleaner.drivers[did].trackpwr:
        cleaner.drivers[did].trackpwr[circuit] = 0 #TODO maybe change defaults
    if circuit not in cleaner.drivers[did].constructor.trackpwr:
        cleaner.drivers[did].constructor.trackpwr[circuit] = 0 #TODO maybe change defaults
    if circuit not in cleaner.drivers[did].constructor.engine.trackpwr:
        cleaner.drivers[did].constructor.engine.trackpwr[circuit] = 0 #TODO maybe change defaults
    
    entry = [
        cleaner.drivers[did].pwr,
        cleaner.drivers[did].constructor.pwr, 
        cleaner.drivers[did].constructor.engine.pwr,
        cleaner.drivers[did].trackpwr[circuit],
        cleaner.drivers[did].constructor.trackpwr[circuit],
        cleaner.drivers[did].constructor.engine.trackpwr[circuit]
    ]
    predictedEntrants.append(entry)

Kimi Räikkönen: -0.04261191758451948 -- 0.022120273288859724
Antonio Giovinazzi: 0.11030212851429542 -- 0.022120273288859724
Sebastian Vettel: -0.16174886038775232 -- -0.4989742601529577
Charles Leclerc: -0.16794614613670164 -- -0.4989742601529577
Romain Grosjean: 0.06053776588028771 -- 0.0028089023535064176
Kevin Magnussen: 0.050832913922083944 -- 0.0028089023535064176
Lando Norris: 0.06484987590666927 -- 0.09022785892893072
Carlos Sainz: -0.028049998951730066 -- 0.09022785892893072
Lewis Hamilton: -0.2907539250682432 -- -0.4418111055467367
Valtteri Bottas: -0.26678386013169303 -- -0.4418111055467367
Sergio Pérez: -0.08507325967063854 -- 0.2320137132835157
Lance Stroll: 0.1749024272494189 -- 0.2320137132835157
Pierre Gasly: 0.14282484438824933 -- -0.3548212925789576
Max Verstappen: -0.17985405601874638 -- -0.3548212925789576
Daniel Ricciardo: -0.01781551667719778 -- 0.13285708462130244
Nico Hülkenberg: 0.12507311258499504 -- 0.13285708462130244
Alexander Albon: 0.1180080032039557 -- -

In [12]:
#Linear regression predict
predictedResults = reg.predict(np.array(predictedEntrants))
print(predictedResults)

[ 0.05289775  0.12191595 -0.46964682 -0.48141537  0.04002181  0.03157551
  0.05067602 -0.00746435 -0.6565892  -0.62808493  0.12861463  0.27323586
 -0.08673533 -0.26888576  0.04971188  0.16189811  0.23409506  0.16172002
  1.0388294   1.0297589 ]


In [13]:
#Forest predict
forestResults = forest.predict(np.array(predictedEntrants))
print(forestResults)

[ 0.0164218   0.07186053 -0.50072626 -0.50650516  0.020899    0.020899
  0.16445152  0.03585134 -0.51906159 -0.51689982 -0.00205197  0.24804136
 -0.36799769 -0.4178354   0.05378999  0.28805357  0.04443071  0.09774345
  1.04861203  1.05085681]


In [14]:
driverResults = {} # {did: {position: amount}}
for index, (did, cid) in enumerate(newDrivers.items()):
    print("{0} ({1}): {2}".format(cleaner.drivers[int(did)].name, cleaner.drivers[int(did)].constructor.name, forestResults[index]))
    newDrivers[did] = forestResults[index]
    driverResults[int(did)] = {}
    
for i in range(1000):
    scoreList = predictQualiResults(circuit, newDrivers)
    for i, drivRes in enumerate(scoreList):
        if i not in driverResults[drivRes[0]]:
            driverResults[drivRes[0]][i] = 0
        driverResults[drivRes[0]][i] += 1
        
for did, res in driverResults.items():
    print("{0} ({1}):".format(cleaner.drivers[int(did)].name, cleaner.drivers[int(did)].constructor.name))
    for pos, amount in sorted(res.items(), key=lambda posAmount: posAmount[0]):
        print("\t{0}: {1} %".format(pos + 1, amount / 10.0))
        
with open('../F1PredictWeb/public/predictions.json', 'w') as fp:
    json.dump(driverResults, fp)

Kimi Räikkönen (Alfa Romeo): 0.016421801329024405
Antonio Giovinazzi (Alfa Romeo): 0.07186052577604335
Sebastian Vettel (Ferrari): -0.5007262578524848
Charles Leclerc (Ferrari): -0.5065051553228196
Romain Grosjean (Haas F1 Team): 0.020899003570808733
Kevin Magnussen (Haas F1 Team): 0.020899003570808733
Lando Norris (McLaren): 0.1644515225005944
Carlos Sainz (McLaren): 0.035851339999689834
Lewis Hamilton (Mercedes): -0.5190615906276094
Valtteri Bottas (Mercedes): -0.5168998157468022
Sergio Pérez (Racing Point): -0.002051974795324301
Lance Stroll (Racing Point): 0.24804136404607519
Pierre Gasly (Red Bull): -0.3679976937390536
Max Verstappen (Red Bull): -0.4178354004000525
Daniel Ricciardo (Renault): 0.05378999073371226
Nico Hülkenberg (Renault): 0.28805357015446503
Alexander Albon (Toro Rosso): 0.04443071390614603
Daniil Kvyat (Toro Rosso): 0.09774344735366887
George Russell (Williams): 1.0486120263788943
Robert Kubica (Williams): 1.0508568147110044
Kimi Räikkönen (Alfa Romeo):
	5: 0.2 %

-0.3754633047458623
-0.04901705829058218
